In [1]:
import numpy as np
import cv2
import os

import keras.backend as k
import keras
from keras.models import Model
from keras.preprocessing import image
from keras.applications import imagenet_utils
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.metrics import categorical_crossentropy
from keras.models import load_model

Using TensorFlow backend.


In [2]:
mobile = keras.applications.mobilenet.MobileNet()

In [3]:
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

In [4]:
x = mobile.layers[-6].output
predictions = Dense(3, activation='softmax')(x)
model = Model(inputs=mobile.inputs, output=predictions)

<ipython-input-4-97e0eef4aeb8>:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  model = Model(inputs=mobile.inputs, output=predictions)


In [5]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [6]:
for layer in model.layers[:-6]:
    layer.trainable = False

In [7]:
path0 = r'E:\tonyStark\NeuralNetworks\gesture\train' #the path to training data folder

train_batch = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).flow_from_directory(path0,
                                                                                                                          target_size=(224,224),
                                                                                                                          batch_size=10)

Found 900 images belonging to 3 classes.


In [8]:
path1 = r'E:\tonyStark\NeuralNetworks\gesture\validation' #the path to validation data folder

validation_batch = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).flow_from_directory(path1,
                                                                                                                          target_size=(224,224),
                                                                                                                          batch_size=10)

Found 270 images belonging to 3 classes.


In [9]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint(r'E:\tonyStark\NeuralNetworks\gesture\train\name.h5',
                            monitor="val_loss",
                            verbose=1,
                            save_best_only=True,
                            mode='min')

earlystop = EarlyStopping(monitor="val_loss",
                         min_delta=0,
                         patience=8,
                         verbose=1,
                         restore_best_weights=True)

reducelr = ReduceLROnPlateau(monitor='val_loss',
                            factor=0.2,
                            patience=2,
                            verbose=1,
                            min_delta=0.0001)

mycallbacks = [earlystop, checkpoint, reducelr]

In [10]:
model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
hist = model.fit_generator(train_batch,
                          steps_per_epoch=30,
                          epochs=10,
                           callbacks=mycallbacks,
                          validation_data=validation_batch,
                          validation_steps=9,
                          verbose=2)

In [ ]:
model.save(r'E:\tonyStark\NeuralNetworks\gesture\best.h5') #save the model